[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Masao-Taketani/JPN-EN-Transformer-translator/blob/master/notebooks/test/train.ipynb)

In [3]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 4.8MB/s 


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
import sentencepiece as spm
import tensorflow as tf

In [13]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64
jpn_sp_model = "drive/My Drive/deep_learning_models/JPN-EN-Transformer-translator/jpn_spm.model"
en_sp_model = "drive/My Drive/deep_learning_models/JPN-EN-Transformer-translator/en_spm.model"
jpn_sp = spm.SentencePieceProcessor()
en_sp = spm.SentencePieceProcessor()
jpn_sp.Load(jpn_sp_model)
en_sp.Load(en_sp_model)

True

In [41]:
def read_data(fpath):
  with open(fpath, "r") as f:
    return f.read()

def get_max_len_and_list(fpath):
  data = read_data(fpath)
  max_len = 0
  li = []
  for line in data.split("\n"):
    li.append(line)
    if max_len < len(line):
      max_len = len(line)
  return max_len, li

In [43]:
jpn_path = "drive/My Drive/deep_learning_models/JPN-EN-Transformer-translator/jpn_data.txt"
en_path = "drive/My Drive/deep_learning_models/JPN-EN-Transformer-translator/en_data.txt"
JPN_MAX_LEN, jpn_data = get_max_len_and_list(jpn_path)
EN_MAX_LEN, en_data = get_max_len_and_list(en_path)
JPN_MAX_LEN, EN_MAX_LEN

(117, 251)

In [44]:
def encode(jpn, en):
  jpn_enc = [jpn_sp.PieceToId("<s>")] + jpn_sp.EncodeAsIds(jpn) + [jpn_sp.PieceToId("</s>")]
  en_enc = [en_sp.PieceToId("<s>")] + en_sp.EncodeAsIds(en) + [en_sp.PieceToId("</s>")]
  return jpn_enc, en_enc

def tf_encode(jpn, en):
  result_jpn, result_en = tf.py_function(encode, [jpn, en], [tf.int64, tf.int64])
  result_jpn.set_shape([None])
  result_en.set_shape([None])
  return result_jpn, result_en

In [45]:
# test
t1, t2 = encode("こんにちは。今日は比較的涼しい日ですね。", "Hello. Today is a relatively cool day, isn't it?")
print(t1)
print(t2)

[1, 5, 101, 126, 10, 211, 6, 4, 2182, 3176, 7892, 314, 5439, 275, 93, 553, 4, 2]
[1, 4241, 4, 1344, 9, 8, 2767, 108, 1625, 103, 13, 9, 42, 10, 19, 29, 20, 2]


In [49]:
jpn_dataset = tf.data.Dataset.from_tensor_slices(jpn_data)
en_dataset = tf.data.Dataset.from_tensor_slices(en_data)
type(jpn_dataset), type(en_dataset)

(tensorflow.python.data.ops.dataset_ops.TensorSliceDataset,
 tensorflow.python.data.ops.dataset_ops.TensorSliceDataset)